# __Search Image by Image__

- Tutorial Difficulty: ★☆☆☆☆
- 7 min read
- Languages: [SQL](https://en.wikipedia.org/wiki/SQL) (100%)
- File location: tutorial_en/thanosql_search/search_image_by_image.ipynb
- References: [Introduction of MNIST DataSet](https://en.wikipedia.org/wiki/MNIST_database), [A Simple Framework for Contrastive Learning of Visual Representations](https://arxiv.org/abs/2002.05709)

## Tutorial Introduction

<div class="admonition note">
    <h4 class="admonition-title">Understanding Image Vectorization</h4>
    <p>Images(height x width x channel [RGB] x color intensity) are meaningless if the information for each pixel is randomly generated. In other words, an image can only be recognized as an image if each pixel has a specific pattern associated with the surrounding pixels. With this information, it can be inferred that an image can be represented on a low-dimensional feature vector. Recently, studies using machine learning to vectorize and express each image in a low-dimensional space based on similarity have been conducted.</p>
</div>

There are several ways to define the similarity of an image. It can refer to the colors being similar, the objects in the image being similar, or the context of the image being similar (ex. a handwritten number). Although it is difficult to give an exact definition of an image similarity, machine learning learns and vectorizes these general features.

ThanoSQL uses the [Self-Supervised Learning Model](https://en.wikipedia.org/wiki/Self-supervised_learning) to input images into the database and retrieve similar images from it. When you upload images to the ThanoSQL's database, a machine learning algorithm places similar images together while placing non-similar images apart. Image features are derived from an unlabeled dataset and fine-tuned with a small amount of labeled data. Then, it can be used for classification or regression tasks.

Furthermore, ThanoSQL uses machine learning algorithms to vectorize datasets. The vectorized data is stored as a database column in the image table and is used to calculate the similarity(distance).

__The following are examples and applications of the ThanoSQL similar image search algorithm.__

- Input your favorite image and have similar artworks searched for and recommended to you.
- Find similar images within an album containing thousands of photos.
- Store your images in the ThanoSQL's database and create your own search engine or machine learning model utilizing the ThanoSQL AutoML regression/classification model.
 
<div class="admonition note">
    <h4 class="admonition-title">In This Tutorial</h4>
    <p>👉 This tutorial will use the MNIST handwriting dataset. Each image consists of a hand written number between 0 and 9 and is correctly labeled. The MNIST handwriting dataset consists of 1,000 train images and 200 test images.</p>
</div>
    
Create a model that uses ThanoSQL to input handwriting data and retrieves similar images from the database.

[![IMAGE](https://docs.thanosql.ai/img/thanosql_search/search_image_by_image/simclr_img7.png "MNIST data") ](https://docs.thanosql.ai/img/thanosql_search/search_image_by_image/simclr_img7.png)

## __0. Prepare Dataset__

As mentioned in the [ThanoSQL Workspace](https://docs.thanosql.ai/en/getting_started/paas/workspace/lab/), you must create an API token and run the query below to execute the query of ThanoSQL. 

In [ ]:
%load_ext thanosql
%thanosql API_TOKEN=<Issued_API_TOKEN>

### __Prepare Dataset__

In [2]:
%%thanosql
GET THANOSQL DATASET mnist_data
OPTIONS (overwrite=True)

Success


<div class="admonition note">
    <h4 class="admonition-title">Query Details</h4>
    <ul>
        <li>"<strong>GET THANOSQL DATASET</strong>" downloads the specified dataset to the workspace.</li>
        <li>"<strong>OPTIONS</strong>" specifies the option values to be used for the <strong>GET THANOSQL DATASET</strong> clause.
        <ul>
            <li>"overwrite": determines whether to overwrite a dataset if it already exists. If set as True, the old dataset is replaced with the new dataset (bool, optional, True|False, default: False)</li>
        </ul>
        </li>
    </ul>
</div>

In [3]:
%%thanosql
COPY mnist_train 
OPTIONS (if_exists='replace')
FROM 'thanosql-dataset/mnist_data/mnist_train.csv'

Success


In [4]:
%%thanosql
COPY mnist_test 
OPTIONS (if_exists='replace')
FROM 'thanosql-dataset/mnist_data/mnist_test.csv'

Success


<div class="admonition note">
    <h4 class="admonition-title">Query Details</h4>
    <ul>
        <li>"<strong>COPY</strong>" specifies the name of the dataset to be saved as a database table.</li>
        <li>"<strong>OPTIONS</strong>" specifies the option values to be used for the <strong>COPY</strong> clause.
        <ul>
           <li>"if_exists": determines how the function should handle the case where the table already exists, it can either raise an error, append to the existing table, or replace the existing table (str, optional, 'fail'|'replace'|'append', default: 'fail')</li>
        </ul>
        </li>
    </ul>
</div>

## __1. Check Dataset__

To create a handwriting classification model, we use the __mnist_train__ table located in the ThanoSQL workspace database. Run the query below to check the contents of the table.

In [5]:
%%thanosql
SELECT * 
FROM mnist_train 
LIMIT 5

,image_path,filename,label
0,thanosql-dataset/mnist_data/train/6782.jpg,6782.jpg,5
1,thanosql-dataset/mnist_data/train/1810.jpg,1810.jpg,5
2,thanosql-dataset/mnist_data/train/33617.jpg,33617.jpg,5
3,thanosql-dataset/mnist_data/train/27802.jpg,27802.jpg,5
4,thanosql-dataset/mnist_data/train/50677.jpg,50677.jpg,5


<div class="admonition note">
    <h4 class="admonition-title">Understanding the Data Table</h4>
    <p>The <strong>mnist_train</strong> table contains the following information.</p>
    <ul>
        <li>image_path: image path</li>
        <li>filename: file name</li>
        <li>label: image label</li>
    </ul>
</div>

## __2. Build an Image Vectorization Model__


To create an image vectorization model with the name __my_image_search_model__ using the __mnist_train__ table, run the following query.  
(Estimated duration of query execution: 1 min)

In [6]:
%%thanosql
BUILD MODEL my_image_search_model
USING SimCLR
OPTIONS (
    image_col='image_path',
    max_epochs=1,
    overwrite=True
    )
AS 
SELECT * 
FROM mnist_train

Success


<div class="admonition note">
    <h4 class="admonition-title">Query Details</h4>
    <ul>
        <li>"<strong>BUILD MODEL</strong>" creates and trains a model named <strong>my_image_search_model</strong>.</li>
        <li>"<strong>USING</strong>" specifies <strong>SimCLR</strong> as the base model.</li>
        <li>"<strong>OPTIONS</strong>" specifies the option values used to create a model.
        <ul>
            <li>"image_col": the name of the column containing the image path (str, default: 'image_path')</li>
            <li>"max_epochs": number of times to train with the training dataset (int, optional, default: 5)</li>
            <li>"overwrite": determines whether to overwrite a model if it already exists. If set as True, the old model is replaced with the new model (bool, optional, True|False, default: False)</li>
        </ul>
        </li>
    </ul>
</div>

To vectorize the __mnist_test__ images run the following "__CONVERT USING__" query. The vectorized results are stored in a user-defined column(default: convert_result) in the __mnist_test__ table.

In [7]:
%%thanosql
CONVERT USING my_image_search_model
OPTIONS (
    image_col='image_path',
    result_col='convert_result'
    )
AS 
SELECT * 
FROM mnist_test

,image_path,filename,label,convert_result
0,thanosql-dataset/mnist_data/test/5099.jpg,5099.jpg,6,"[b'\xd7', b'q', b'&', b'<', b'\xa1', b'<', b'\..."
1,thanosql-dataset/mnist_data/test/9239.jpg,9239.jpg,6,"[b'\x1e', b'@', b'\xa9', b'=', b']', b'K', b'\..."
2,thanosql-dataset/mnist_data/test/2242.jpg,2242.jpg,6,"[b'\x87', b'\xd1', b' ', b'>', b'\x83', b'e', ..."
3,thanosql-dataset/mnist_data/test/3451.jpg,3451.jpg,6,"[b'\xac', b'\xd2', b""'"", b'>', b'*', b'\xfa', ..."
4,thanosql-dataset/mnist_data/test/2631.jpg,2631.jpg,6,"[b'O', b'\xf1', b'M', b'=', b'9', b'w', b'?', ..."
...,...,...,...,...
195,thanosql-dataset/mnist_data/test/8045.jpg,8045.jpg,8,"[b'\x00', b'\x00', b'\x00', b'\x00', b'\xa4', ..."
196,thanosql-dataset/mnist_data/test/9591.jpg,9591.jpg,8,"[b'\x99', b'\x15', b'9', b'>', b'\xb7', b'\x88..."
197,thanosql-dataset/mnist_data/test/7425.jpg,7425.jpg,8,"[b'\x1c', b'H', b'\x8a', b'>', b'f', b'\x83', ..."
198,thanosql-dataset/mnist_data/test/2150.jpg,2150.jpg,8,"[b'\xf4', b'Y', b'S', b'>', b'\x96', b'^', b'\..."


<div class="admonition note">
    <h4 class="admonition-title">Query Details</h4>
    <ul>
        <li>"<strong>CONVERT USING</strong>" uses <strong>my_image_search_model</strong> as an algorithm for image vectorizaion.   </li>
        <li>"<strong>OPTIONS</strong>" specifies the options to be used for image vectorization.
        <ul>
            <li>"image_col": the name of the column containing the image path (str, default: 'image_path')</li>
            <li>"result_col": defines the column name that contains the vectorized results (str, optional, default: 'convert_result')</li>
        </ul>
        </li>
    </ul>
</div>

Execute the "__CONVERT USING__" query statement below and save the converted result in a new table so that it can be used with other __ThanoSQL__ query statements.

In [10]:
%%thanosql
CREATE TABLE mnist_test_convert_en AS 
SELECT * FROM ( 
    CONVERT USING my_image_search_model
    OPTIONS (
        image_col='image_path',
        result_col='convert_result'
        )
    AS 
    SELECT * 
    FROM mnist_test
)

Success


## __3. Search for Similar Images Using Image Vectorization Models__

This step uses the __my_image_search_model__ image vectorization model and the test table to search for images similar to the "923.jpg" image(handwritten 8).

<a href="https://docs.thanosql.ai/img/thanosql_search/search_image_by_image/simclr_img8.png">
    <img alt="IMAGE" src="https://docs.thanosql.ai/img/thanosql_search/search_image_by_image/simclr_img8.png" style="width:100px">
</a>

<p style="text-align:center">923.jpg Image File </p>

In [11]:
%%thanosql
SEARCH IMAGE
USING my_image_search_model
OPTIONS (
    search_by='image',
    search_input='thanosql-dataset/mnist_data/test/923.jpg',
    emb_col='convert_result',
    result_col='search_result'
    )
AS
SELECT * 
FROM mnist_test_convert_en

,image_path,filename,label,convert_result,search_result
0,thanosql-dataset/mnist_data/test/923.jpg,923.jpg,8,"[b'\xcd', b'\x1d', b'\x08', b'>', b'\xf6', b'\...",1.000000
1,thanosql-dataset/mnist_data/test/7645.jpg,7645.jpg,8,"[b']', b'\xac', b'L', b'=', b'\xe2', b'\x99', ...",0.997068
2,thanosql-dataset/mnist_data/test/5087.jpg,5087.jpg,8,"[b'\xc8', b'\x80', b'o', b'=', b'\xe6', b'\xdc...",0.996402
3,thanosql-dataset/mnist_data/test/685.jpg,685.jpg,8,"[b'>', b'\x84', b';', b'=', b'\xb6', b'N', b'\...",0.995973
4,thanosql-dataset/mnist_data/test/6573.jpg,6573.jpg,8,"[b'\x94', b'\xf7', b'\xfe', b'=', b'\x11', b'\...",0.995699
...,...,...,...,...,...
195,thanosql-dataset/mnist_data/test/2220.jpg,2220.jpg,4,"[b'\xfc', b'6', b'\x16', b'>', b'\xa8', b'\x99...",0.984654
196,thanosql-dataset/mnist_data/test/3684.jpg,3684.jpg,3,"[b's', b'\x00', b'\xd1', b'=', b'\xa0', b'\x97...",0.984400
197,thanosql-dataset/mnist_data/test/7020.jpg,7020.jpg,1,"[b'\x15', b'\xe4', b'\xf1', b'<', b'n', b'\x86...",0.984273
198,thanosql-dataset/mnist_data/test/2938.jpg,2938.jpg,1,"[b'h', b'\xdc', b'\xb0', b'=', b'\xee', b'\x1d...",0.983866


<div class="admonition note">
    <h4 class="admonition-title">Query Details</h4>
    <ul>
        <li>"<strong>SEARCH IMAGE [image|text|audio|video]</strong>" defines the image|text|audio|video file type to search for.</li>
        <li>"<strong>USING</strong>" defines the model used for image vectorization.</li>
        <li>"<strong>OPTIONS</strong>" specifies the options to be used for image searching.
        <ul>
            <li>"search_by": defines the image|text|audio|video type to be used for the search (str)</li>
            <li>"search_input": defines the input to be used for the search (str)</li>
            <li>"emb_col": the column that contains the vectorized results (str)</li>
            <li>"result_col": defines the name of the column that contains the search results (str, optional, default: 'search_result')</li>
        </ul>
        </li>
        <li>"<strong>AS</strong>" defines the embedding table to be used for searches. In this example, the <strong>mnist_test</strong> table is used. </li>
    </ul>
</div>

To output the "__SEARCH__" result using the "__PRINT__" clause to output the top four most similar images, run the following query. Though we've only done a minimal amount of training, you can see that images similar to 8 are returned.

In [12]:
%%thanosql
PRINT IMAGE 
AS (
    SELECT image_path, search_result 
    FROM (
        SEARCH IMAGE 
        USING my_image_search_model 
        OPTIONS (
            search_by='image',
            search_input='thanosql-dataset/mnist_data/test/923.jpg',
            emb_col='convert_result',
            result_col='search_result',
            top_k=4
            )
        AS 
        SELECT * 
        FROM mnist_test_convert_en
        )
    )

/home/jovyan/thanosql-dataset/mnist_data/test/923.jpg


/home/jovyan/thanosql-dataset/mnist_data/test/7645.jpg


/home/jovyan/thanosql-dataset/mnist_data/test/5087.jpg


/home/jovyan/thanosql-dataset/mnist_data/test/685.jpg


<div class="admonition danger">
    <h4 class="admonition-title">Note</h4>
    <p>The training options of the algorithm recognize the image regardless of the image's left-right inversion and color differences. This is because a dog's picture should be recognized as a dog even if it is flipped or has a color difference. If the color feature is important, such as clothing images, or if vertical and horizontal inversions are important, such as numbers, the training options should be changed.</p>
</div>

## __4. In Conclusion__

In this tutorial, we used the MNIST handwriting dataset to vectorize images and perform image search. As this is a beginner-level tutorial, we focused on the process rather than accuracy. The model's accuracy can be improved by adding precise tuning and small amounts of labeling to each dataset.

* [How to Upload My Data to the ThanoSQL Workspace](https://docs.thanosql.ai/en/getting_started/data_upload/)
* [How to Create a Table Using My Data](https://docs.thanosql.ai/en/how-to_guides/ThanoSQL_query/COPY_SYNTAX/)
* [How to Upload My Model to the ThanoSQL Workspace](https://docs.thanosql.ai/en/how-to_guides/ThanoSQL_query/UPLOAD_MODEL_SYNTAX/)

<div class="admonition tip">
    <h4 class="admonition-title">Inquiries About Deploying a Model for Your Own Service</h4>
    <p>If you have any difficulties creating your own model using ThanoSQL or applying it to your services, please feel free to contact us below😊</p>
    <p>For inquiries regarding building an image similarity search models: <a href="mailto:contact@smartmind.team">contact@smartmind.team</a></p>
</div>